# Corona Development Germany EDA V1📊⚡

This notebook is based on the COVID-19 Tracking Germany and is part of a global initiative to fight Covid. The notebook presents 3 graphs so far:

1. The daily new infections in each state
2. The accumulated infections in each state
3. The 7-day incidence rate, which I find most exciting

In the next version, I will examine the incidence figures for each district and what government action has had which effect on the incidence rate. 

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import geopandas as gpd
import seaborn as sns # fancy statistics plots
import matplotlib as mpl
import datetime
import plotly.graph_objects as go
import matplotlib.pyplot as plt
from urllib.request import urlopen
import json
import collections
import plotly.express as px
import os
import pickle

In [ ]:
path_to_file_covid = '../input/covid19-tracking-germany/covid_de.csv'
path_to_file_demo  = '../input/covid19-tracking-germany/demographics_de.csv'
path_to_file_shape = '../input/covid19-tracking-germany/de_state.shp'

In [ ]:
#getting the data
covid_de  = pd.read_csv(path_to_file_covid, index_col="date", parse_dates=True) #cases and deaths per state and age and sex
demo_de   = pd.read_csv(path_to_file_demo)    # demography file
shape_de2 = gpd.read_file(path_to_file_shape) # geography file

Bundesländer:

In [ ]:
# replace Umlaute
shape_tmp = shape_de2.replace({'Baden-Württemberg' : 'Baden-Wuerttemberg', 'Thüringen' : 'Thueringen' }).copy()
shape_de = shape_tmp.rename(columns={'GEN': 'state'}).copy()

# Plotting Covid-19 data on German Map using Plotly Choropleth:

In [ ]:
covid_de  = pd.read_csv(path_to_file_covid, index_col="date", parse_dates=True) #cases and deaths per state and age and sex

In [ ]:
# max_colour = max(array_discovered_Cases)
# min_colour = min(array_discovered_Cases)
# cmap = cm.linear.YlOrRd_09.scale(min_colour, max_colour)
# states_df['colour'] = array_discovered_Cases.map(cmap)

In [ ]:

# covid_de_sorted  =covid_de.sort_values(['state','county','age_group']).sort_index(kind='mergesort')
# # covid_de_sorted = covid_de_sorted.sort_index()
# covid_de_sorted[2000:2002]

In [ ]:
# #Define a colour map in terms of the log of the number of confirmed cases
# max_colour = max(covid_de_sorted['log_Confirmed'])
# min_colour = min(covid_de_sorted['log_Confirmed'])
# cmap = cm.linear.YlOrRd_09.scale(0, max_colour)
# covid_de_sorted['colour'] = covid_de_sorted['log_Confirmed'].map(cmap)

In [ ]:
# All states in the Kaggle Dataset
np.union1d(np.array(covid_de['state']),[])

In [ ]:
with urlopen('https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/master/2_bundeslaender/4_niedrig.geo.json') as response:
    geojson = json.load(response)

# All states in the geojson
States = [] 
for obj in geojson['features']:
    States.append(obj['properties']['name'])
States

In [ ]:
states = np.union1d(np.array(covid_de['state']),[])

GeoStateDict = {
    'Baden-Wuerttemberg':0, 
    'Bayern':1, 
    'Berlin':2, 
    'Brandenburg':3, 
    'Bremen':4,
    'Hamburg':5, 
    'Hessen':6, 
    'Mecklenburg-Vorpommern':7, 
    'Niedersachsen':8,
    'Nordrhein-Westfalen':9, 
    'Rheinland-Pfalz':10, 
    'Saarland':11, 
    'Sachsen':12,
    'Sachsen-Anhalt':13, 
    'Schleswig-Holstein':14, 
    'Thueringen':15
}

InhabitansStateDict = {
    'Baden-Wuerttemberg':111.0, 
    'Bayern':131.25 , 
    'Berlin':36.69 , 
    'Brandenburg':25.22 , 
    'Bremen':6.81 ,
    'Hamburg':18.47  , 
    'Hessen':62.88 , 
    'Mecklenburg-Vorpommern':16.08 , 
    'Niedersachsen':79.94 ,
    'Nordrhein-Westfalen':179.47 , 
    'Rheinland-Pfalz':40.94 , 
    'Saarland':9.87, 
    'Sachsen':40.72,
    'Sachsen-Anhalt':21.95, 
    'Schleswig-Holstein':29.04, 
    'Thueringen':21.33    
}
#Source:
#https://de.statista.com/statistik/daten/studie/71085/umfrage/verteilung-der-einwohnerzahl-nach-bundeslaendern/

In [ ]:
# Get the summed up cases for provided date for every state and return it as a dataframe
def getCovidSum(date,input_covid_de_sorted):
    
    cases_of_state = []
    state_ids = []
    dates = []
    dates_str = []
    CasesPer100k = []
    for state in states:
        cases_of_state.append(np.sum(
            input_covid_de_sorted.loc['{}'.format(date)].query("state == '{}'".format(state))['cases']))
        state_ids.append(GeoStateDict[state])
        dates.append(date)
        dates_str.append(str(date.strftime("%Y-%m-%d")))
        CasesPer100k.append(np.sum(input_covid_de_sorted.loc['{}'.format(date)].query("state == '{}'".format(state))['cases'])/InhabitansStateDict[state])
        
    
    d = {'date': dates,'date_str': dates_str,'state': states,'state_id' : state_ids, 'cases': cases_of_state,'NewCasesPer100k':CasesPer100k}
    df_germany = pd.DataFrame(data=d)
    return df_germany

In [ ]:
# create combined dataframe of all days
current_State_data=  getCovidSum(datetime.date(2020, 3, 26),covid_de)

start_date = datetime.date(2020, 3, 27)
end_date = datetime.date(2021, 1, 1)
delta = datetime.timedelta(days=1)

delta5 = datetime.timedelta(days=5)




while start_date <= end_date:
    start_date += delta

    # Data of the day
    time_data = getCovidSum(start_date,covid_de)
    time_data["All_Cases"] = 0
    time_data["7TageInzidenz"] = 0
    
    #get The Amount Of All Cases That have been noticed before
    for index, row in time_data.iterrows():
        # get all the previous cases
        time_data["All_Cases"].iloc[index] = np.sum(current_State_data.query("state == '{}'".format(row['state']))['cases'])
    
    #Get The Amount of NewCases Per 7 Days
    for index, row in time_data.iterrows():
        # get all the previous cases
#         test = current_State_data[current_State_data.date > row['date'] - pd.to_timedelta("5day")]
#         time_data["7TageInzidenz"].iloc[index] = np.sum(current_State_data.query("state == '{}'".format(row['state']))  ['NewCasesPer100k'])
 
        time_data["7TageInzidenz"].iloc[index] = np.sum(current_State_data[current_State_data.date > row['date'] - pd.to_timedelta("7day")].query("state == '{}'".format(row['state'])) ['NewCasesPer100k'])
    
    current_State_data = current_State_data.append(time_data).reset_index(drop=True)



    
    

In [ ]:
#Transform confirmed cases to log10 scale to reduce the scale of plotting.
# covid_de['log_Confirmed'] = np.log10(covid_de['cases'])
# covid_de.head(1)

In [ ]:
# use current_State_data as new output 
os.mkdir('./preprocessing')
with open('./preprocessing/current_State_data.pickle', 'wb') as f:
    pickle.dump(current_State_data, f)



# Daily New Cases in each State

In [ ]:
# fig_1 = px.choropleth(current_State_data, geojson=geojson, color="cases",
#                     locations="state_id",color_continuous_scale="Plasma",
#                     animation_frame ="date_str",
#                     hover_data=['state','cases']
                    
#                    )
# fig_1.update_geos(fitbounds="locations")
# fig_1.update_layout(title='Germany new cases',height=800)
# fig_1.show()

In [ ]:
#fig_1.write_html("images/file1.html")

# Summed Up Daily New Cases in each State

In [ ]:
# fig_2 = px.choropleth(current_State_data, geojson=geojson, color="All_Cases",
#                     locations="state_id",color_continuous_scale="Plasma",
#                     animation_frame ="date_str",
#                     hover_data=['state','All_Cases']
                    
#                    )
# fig_2.update_geos(fitbounds="locations")
# fig_2.update_layout(title='Germany total cases map - summed up',height=800)
# fig_2.show()

# 7-day incidence map

In [ ]:
# fig_3 = px.choropleth(current_State_data, geojson=geojson, color="7TageInzidenz",
#                     locations="state_id",color_continuous_scale="Plasma",
#                     animation_frame ="date_str",
#                     range_color=[0,400],
#                     hover_data=['state','7TageInzidenz','All_Cases'],
                    
#                    )
# fig_3.update_geos(fitbounds="locations")
# fig_3.update_layout(title='7-day incidence map complete time',height=800)
# fig_3.show()

In [ ]:
# fig = px.choropleth(current_State_data[:800], geojson=geojson, color="7TageInzidenz",
#                     locations="state_id",color_continuous_scale="Plasma",
#                     animation_frame ="date_str",
#                     range_color=[0,100],
#                     hover_data=['state','7TageInzidenz','All_Cases'],
                    
#                    )
# fig.update_geos(fitbounds="locations")
# fig.update_layout(title='7-day incidence map spring',height=800)
# fig.show()

# Landkreise 

In [ ]:
with urlopen('https://raw.githubusercontent.com/isellsoap/deutschlandGeoJSON/master/4_kreise/3_mittel.geo.json') as response:
    geojson_Landkreise = json.load(response)


In [ ]:
# All states in the Kaggle Dataset
LandkreiseDf = np.union1d(np.array(covid_de['county']),[])
for Landkreis in LandkreiseDf:
    
    Landkreis_new = Landkreis.replace('LK','')
    Landkreis_new = Landkreis_new.replace('SK','')
    if Landkreis_new[0] == ' ':
        Landkreis_new = Landkreis_new[1:]
    #print(Landkreis_new)
    print(Landkreis)
 

In [ ]:
# All states in the geojson
LandkreiseGeoJson = [] 
for i, obj in enumerate(geojson_Landkreise['features']):
    LandkreiseGeoJson.append(obj['properties']['NAME_3'])
    Landreis_print = obj['properties']['NAME_3']
    Landreis_print = Landreis_print.replace('ü','ue')
    Landreis_print = Landreis_print.replace('ä','ae')
    Landreis_print = Landreis_print.replace('ö','oe')
    Landreis_print = Landreis_print.replace('ß','ss')
    Landreis_print = Landreis_print.replace(' Staedte','')
    
    print(i, Landreis_print)


In [ ]:
LandkreiseGeoJson

In [ ]:
# Source:
# https://de.wikipedia.org/wiki/Liste_der_Landkreise_in_Deutschland
# https://de.wikipedia.org/wiki/Liste_der_kreisfreien_St%C3%A4dte_in_Deutschland

InhabitansLandkreiseDict = {

'LK Ahrweiler' :130086,
'LK Aichach-Friedberg' :134655,
'LK Alb-Donau-Kreis' :197076,
'LK Altenburger Land' :89393,
'LK Altenkirchen' :128805,
'LK Altmarkkreis Salzwedel' :83173,
'LK Altoetting' :111516,
'LK Alzey-Worms' :129687,
'LK Amberg-Sulzbach' :103049,
'LK Ammerland' :124859,
'LK Anhalt-Bitterfeld' :158486,
'LK Ansbach' :184591,
'LK Aschaffenburg' :174200,
'LK Augsburg' :253468,
'LK Aurich' :189694,
'LK Bad Duerkheim' :132671,
'LK Bad Kissingen' :103235,
'LK Bad Kreuznach' :158345,
'LK Bad Toelz-Wolfratshausen' :127917,
'LK Bamberg' :147163,
'LK Barnim' :185244,
'LK Bautzen' :299758,
'LK Bayreuth' :103664,
'LK Berchtesgadener Land' :105929,
'LK Bergstrasse' :270340,
'LK Bernkastel-Wittlich' :112483,
'LK Biberach' :201282,
'LK Birkenfeld' :80951,
'LK Bitburg-Pruem' :99100,
'LK Bodenseekreis' :217470,
'LK Boeblingen' :392807,
'LK Boerde' :170923,
'LK Borken' :371339,
'LK Breisgau-Hochschwarzwald' :263601,
'LK Burgenlandkreis' :178846,
'LK Calw' :159201,
'LK Celle' :179011,
'LK Cham' :127998,
'LK Cloppenburg' :170682,
'LK Coburg' :86747,
'LK Cochem-Zell' :61375,
'LK Coesfeld' :220586,
'LK Cuxhaven' :198038,
'LK Dachau' :154899,
'LK Dahme-Spreewald' :170791,
'LK Darmstadt-Dieburg' :297844,
'LK Deggendorf' :119478,
'LK Diepholz' :217089,
'LK Dillingen a.d.Donau' :96562,
'LK Dingolfing-Landau' :96683,
'LK Dithmarschen' :133193,
'LK Donau-Ries' :133783,
'LK Donnersbergkreis' :75325,
'LK Dueren' :264638,
'LK Ebersberg' :143649,
'LK Eichsfeld' :100006,
'LK Eichstaett' :132881,
'LK Elbe-Elster' :101827,
'LK Emmendingen' :166408,
'LK Emsland' :326954,
'LK Ennepe-Ruhr-Kreis' :324106,
'LK Enzkreis' :199556,
'LK Erding' :138182,
'LK Erlangen-Hoechstadt' :137262,
'LK Erzgebirgskreis' :334948,
'LK Esslingen' :535024,
'LK Euskirchen' :193656,
'LK Forchheim' :116203,
'LK Freising' :180007,
'LK Freudenstadt' :118243,
'LK Freyung-Grafenau' :78362,
'LK Friesland' :98704,
'LK Fuerstenfeldbruck' :219311,
'LK Fuerth' :117853,
'LK Fulda' :223145,
'LK Garmisch-Partenkirchen' :88424,
'LK Germersheim' :129013,
'LK Giessen' :270688,
'LK Gifhorn' :176523,
'LK Goeppingen' :258145,
'LK Goerlitz' :252725,
'LK Goettingen' :326041,
'LK Goslar' :136292,
'LK Gotha' :134908,
'LK Grafschaft Bentheim' :137162,
'LK Greiz' :97398,
'LK Gross-Gerau' :275726,
'LK Guenzburg' :127027,
'LK Guetersloh' :364938,
'LK Hameln-Pyrmont' :148549,
'LK Harburg' :254431,
'LK Harz' :213310,
'LK Hassberge' :84384,
'LK Havelland' :162996,
'LK Heidekreis' :140673,
'LK Heidenheim' :132777,
'LK Heilbronn' :344456,
'LK Heinsberg' :255555,
'LK Helmstedt' :91297,
'LK Herford' :250578,
'LK Hersfeld-Rotenburg' :120719,
'LK Herzogtum Lauenburg' :198019,
'LK Hildburghausen' :63197,
'LK Hildesheim' :275817,
'LK Hochsauerlandkreis' :259777,
'LK Hochtaunuskreis' :236914,
'LK Hoexter' :31288,
'LK Hof' :94801,
'LK Hohenlohekreis' :112655,
'LK Holzminden' :70458,
'LK Ilm-Kreis' :106249,
'LK Jerichower Land' :89589,
'LK Kaiserslautern' :105979,
'LK Karlsruhe' :445101,
'LK Kassel' :236764,
'LK Kelheim' :123058,
'LK Kitzingen' :91155,
'LK Kleve' :312465,
'LK Konstanz' :286305,
'LK Kronach' :66743,
'LK Kulmbach' :71566,
'LK Kusel' :70219,
'LK Kyffhaeuserkreis' :74212,
'LK Lahn-Dill-Kreis' :253319,
'LK Landsberg a.Lech' :120302,
'LK Landshut' :159895,
'LK Leer' :170756,
'LK Leipzig' :258139,
'LK Lichtenfels' :66776,
'LK Limburg-Weilburg' :171912,
'LK Lindau' :81981,
'LK Lippe' :347514,
'LK Loerrach' :228736,
'LK Ludwigsburg' :545423,
'LK Ludwigslust-Parchim' :211779,
'LK Luechow-Dannenberg' :48424,
'LK Lueneburg' :184139,
'LK Maerkisch-Oderland' :195751,
'LK Maerkischer Kreis' :410222,
'LK Main-Kinzig-Kreis' :420552,
'LK Main-Spessart' :126158,
'LK Main-Tauber-Kreis' :132399,
'LK Main-Taunus-Kreis' :238558,
'LK Mainz-Bingen' :211417,
'LK Mansfeld-Suedharz' :134942,
'LK Marburg-Biedenkopf' :247084,
'LK Mayen-Koblenz' :214434,
'LK Mecklenburgische Seenplatte' :258074,
'LK Meissen' :241717,
'LK Merzig-Wadern' :103243,
'LK Mettmann' :485570,
'LK Miesbach' :100010,
'LK Miltenberg' :128743,
'LK Minden-Luebbecke' :310409,
'LK Mittelsachsen' :304099,
'LK Muehldorf a.Inn' :115872,
'LK Muenchen' :350473,
'LK Neckar-Odenwald-Kreis' :143633,
'LK Neu-Ulm' :175204,
'LK Neuburg-Schrobenhausen' :97303,
'LK Neumarkt i.d.OPf.' :134573,
'LK Neunkirchen' :131408,
'LK Neustadt a.d.Aisch-Bad Windsheim' :101014,
'LK Neustadt a.d.Waldnaab' :94450,
'LK Neuwied' :182811,
'LK Nienburg (Weser)' :121390,
'LK Nordfriesland' :165951,
'LK Nordhausen' :83416,
'LK Nordsachsen' :197741,
'LK Nordwestmecklenburg' :157322,
'LK Northeim' :132285,
'LK Nuernberger Land' :170792,
'LK Oberallgaeu' :156008,
'LK Oberbergischer Kreis' :272057,
'LK Oberhavel' :212914,
'LK Oberspreewald-Lausitz' :109371,
'LK Odenwaldkreis' :96703,
'LK Oder-Spree' :178803,
'LK Offenbach' :355813,
'LK Oldenburg' :130890,
'LK Olpe' :133955,
'LK Ortenaukreis' :430953,
'LK Osnabrueck' :358080,
'LK Ostalbkreis' :314025,
'LK Ostallgaeu' :141182,
'LK Osterholz' :113928,
'LK Ostholstein' :200539,
'LK Ostprignitz-Ruppin' :98861,
'LK Paderborn' :307839,
'LK Passau' :192656,
'LK Peine' :134801,
'LK Pfaffenhofen a.d.Ilm' :128227,
'LK Pinneberg' :316103,
'LK Ploen' :128686,
'LK Potsdam-Mittelmark' :216566,
'LK Prignitz' :76158,
'LK Rastatt' :231420,
'LK Ravensburg' :285424,
'LK Recklinghausen' :614137,
'LK Regen' :77410,
'LK Regensburg' :194070,
'LK Rems-Murr-Kreis' :427248,
'LK Rendsburg-Eckernfoerde' :274098,
'LK Reutlingen' :287034,
'LK Rhein-Erft-Kreis' :470615,
'LK Rhein-Hunsrueck-Kreis' :103163,
'LK Rhein-Kreis Neuss' :451730,
'LK Rhein-Lahn-Kreis' :122297,
'LK Rhein-Neckar-Kreis' :548355,
'LK Rhein-Pfalz-Kreis' :154609,
'LK Rhein-Sieg-Kreis' :600764,
'LK Rheingau-Taunus-Kreis' :187160,
'LK Rheinisch-Bergischer Kreis' :283271,
'LK Rhoen-Grabfeld' :79635,
'LK Rosenheim' :261330,
'LK Rostock' :215794,
'LK Rotenburg (Wuemme)' :163782,
'LK Roth' :126749,
'LK Rottal-Inn' :121502,
'LK Rottweil' :139878,
'LK Saale-Holzland-Kreis' :82950,
'LK Saale-Orla-Kreis' :80312,
'LK Saalekreis' :183815,
'LK Saalfeld-Rudolstadt' :103199,
'LK Saar-Pfalz-Kreis' :142196,
'LK Saarlouis' :194319,
'LK Saechsische Schweiz-Osterzgebirge' :245586,
'LK Salzlandkreis' :189125,
'LK Sankt Wendel' :26582,
'LK Schaumburg' :157820,
'LK Schleswig-Flensburg' :201156,
'LK Schmalkalden-Meiningen' :124916,
'LK Schwaebisch Hall' :196761,
'LK Schwalm-Eder-Kreis' :179673,
'LK Schwandorf' :147872,
'LK Schwarzwald-Baar-Kreis' :212506,
'LK Schweinfurt' :115445,
'LK Segeberg' :277175,
'LK Siegen-Wittgenstein' :276944,
'LK Sigmaringen' :130849,
'LK Soemmerda' :69427,
'LK Soest' :301785,
'LK Sonneberg' :57717,
'LK Spree-Neisse' :113720,
'LK Stade' :204512,
'LK Stadtverband Saarbruecken' :328700,
'LK Starnberg' :136667,
'LK Steinburg' :131013,
'LK Steinfurt' :448220,
'LK Stendal' :111190,
'LK Stormarn' :244156,
'LK Straubing-Bogen' :101120,
'LK Suedliche Weinstrasse' :110521,
'LK Suedwestpfalz' :94831,
'LK Teltow-Flaeming' :169997,
'LK Tirschenreuth' :72046,
'LK Traunstein' :177319,
'LK Trier-Saarburg' :149398,
'LK Tuebingen' :228678,
'LK Tuttlingen' :140766,
'LK Uckermark' :118947,
'LK Uelzen' :92389,
'LK Unna' :394891,
'LK Unstrut-Hainich-Kreis' :102232,
'LK Unterallgaeu' :145341,
'LK Vechta' :142814,
'LK Verden' :137133,
'LK Viersen' :298863,
'LK Vogelsbergkreis' :105643,
'LK Vogtlandkreis' :225997,
'LK Vorpommern-Greifswald' :235623,
'LK Vorpommern-Ruegen' :224702,
'LK Vulkaneifel' :60646,
'LK Waldeck-Frankenberg' :156406,
'LK Waldshut' :171003,
'LK Warendorf' :277840,
'LK Wartburgkreis' :118974,
'LK Weilheim-Schongau' :135478,
'LK Weimarer Land' :82156,
'LK Weissenburg-Gunzenhausen' :94734,
'LK Werra-Meissner-Kreis' :100629,
'LK Wesel' :459976,
'LK Wesermarsch' :88583,
'LK Westerwaldkreis' :201904,
'LK Wetteraukreis' :308339,
'LK Wittenberg' :124953,
'LK Wittmund' :56926,
'LK Wolfenbuettel' :119622,
'LK Wuerzburg' :162302,
'LK Wunsiedel i.Fichtelgebirge' :72655,
'LK Zollernalbkreis' :189363,
'LK Zwickau' :315002,
'Region Hannover' :536925,
'SK Amberg' :42207,
'SK Ansbach' :41798,
'SK Aschaffenburg' :71002,
'SK Augsburg' :296582,
'SK Baden-Baden' :55185,
'SK Bamberg' :77373,
'SK Bayreuth' :74783,
'SK Berlin Charlottenburg-Wilmersdorf' :3669491,
'SK Berlin Friedrichshain-Kreuzberg' :1,
'SK Berlin Lichtenberg' :1,
'SK Berlin Marzahn-Hellersdorf' :1,
'SK Berlin Mitte' :1,
'SK Berlin Neukoelln' :1,
'SK Berlin Pankow' :1,
'SK Berlin Reinickendorf' :1,
'SK Berlin Spandau' :1,
'SK Berlin Steglitz-Zehlendorf' :1,
'SK Berlin Tempelhof-Schoeneberg' :1,
'SK Berlin Treptow-Koepenick' :1,
'SK Bielefeld' :334195,
'SK Bochum' :365587,
'SK Bonn' :329673,
'SK Bottrop' :117565,
'SK Brandenburg a.d.Havel' :72184,
'SK Braunschweig' :249406,
'SK Bremen' :567559,
'SK Bremerhaven' :113643,
'SK Chemnitz' :246334,
'SK Coburg' :41072,
'SK Cottbus' :99678,
'SK Darmstadt' :159878,
'SK Delmenhorst' :77559,
'SK Dessau-Rosslau' :80103,
'SK Dortmund' :588250,
'SK Dresden' :556780,
'SK Duesseldorf' :498686,
'SK Duisburg' :621877,
'SK Eisenach' :42250,
'SK Emden' :49913,
'SK Erfurt' :213981,
'SK Erlangen' :112528,
'SK Essen' :582760,
'SK Flensburg' :90164,
'SK Frankenthal' :48762,
'SK Frankfurt (Oder)' :57751,
'SK Frankfurt am Main' :763380,
'SK Freiburg i.Breisgau' :231195,
'SK Fuerth' :128497,
'SK Gelsenkirchen' :259645,
'SK Gera' :93125,
'SK Hagen' :188686,
'SK Halle' :238762,
'SK Hamburg' :1841179,
'SK Hamm' :179916,
'SK Heidelberg' :161485,
'SK Heilbronn' :126592,
'SK Herne' :156449,
'SK Hof' :45825,
'SK Ingolstadt' :137392,
'SK Jena' :111343,
'SK Kaiserslautern' :100030,
'SK Karlsruhe' :312060,
'SK Kassel' :202137,
'SK Kaufbeuren' :44398,
'SK Kempten' :69151,
'SK Kiel' :246794,
'SK Koblenz' :114052,
'SK Koeln' :1087863,
'SK Krefeld' :227417,
'SK Landau i.d.Pfalz' :46881,
'SK Landshut' :73411,
'SK Leipzig' :593145,
'SK Leverkusen' :163729,
'SK Ludwigshafen' :216530,
'SK Luebeck' :172253,
'SK Magdeburg' :237565,
'SK Mainz' :218578,
'SK Mannheim' :310658,
'SK Memmingen' :44100,
'SK Moenchengladbach' :261034,
'SK Muelheim a.d.Ruhr' :170632,
'SK Muenchen' :1484226,
'SK Muenster' :315293,
'SK Neumuenster' :80196,
'SK Neustadt a.d.Weinstrasse' :53264,
'SK Nuernberg' :518370,
'SK Oberhausen' :210764,
'SK Offenbach' :130280,
'SK Oldenburg' :169077,
'SK Osnabrueck' :165251,
'SK Passau' :52803,
'SK Pforzheim' :125957,
'SK Pirmasens' :40231,
'SK Potsdam' :180334,
'SK Regensburg' :153094,
'SK Remscheid' :111338,
'SK Rosenheim' :63551,
'SK Rostock' :209191,
'SK Salzgitter' :104291,
'SK Schwabach' :40981,
'SK Schweinfurt' :53426,
'SK Schwerin' :95653,
'SK Solingen' :159245,
'SK Speyer' :50561,
'SK Straubing' :47791,
'SK Stuttgart' :635911,
'SK Suhl' :36789,
'SK Trier' :111528,
'SK Ulm' :126790,
'SK Weiden i.d.OPf.' :42743,
'SK Weimar' :65228,
'SK Wiesbaden' :278474,
'SK Wilhelmshaven' :76089,
'SK Wolfsburg' :124371,
'SK Worms' :83542,
'SK Wuerzburg' :355100,
'SK Wuppertal' :127934,
'SK Zweibruecken' :34193,
'StadtRegion Aachen' :248960,


}


In [ ]:
# Map LankreiseDf to corresponding id in GeoJson 

# Done via using Excel VLOOKUP 

GeoDictLandkreise = {

'LK Ahrweiler' :304,
'LK Aichach-Friedberg' :131,
'LK Alb-Donau-Kreis' :21,
'LK Altenburger Land' :412,
'LK Altenkirchen' :305,
'LK Altmarkkreis Salzwedel' :359,
'LK Altoetting' :85,
'LK Alzey-Worms' :315,
'LK Amberg-Sulzbach' :121,
'LK Ammerland' :251,
'LK Anhalt-Bitterfeld' :347,
'LK Ansbach' :30,
'LK Aschaffenburg' :145,
'LK Augsburg' :132,
'LK Aurich' :252,
'LK Bad Duerkheim' :316,
'LK Bad Kissingen' :147,
'LK Bad Kreuznach' :306,
'LK Bad Toelz-Wolfratshausen' :86,
'LK Bamberg' :108,
'LK Barnim' :158,
'LK Bautzen' :381,
'LK Bayreuth' :110,
'LK Berchtesgadener Land' :87,
'LK Bergstrasse' :178,
'LK Bernkastel-Wittlich' :334,
'LK Biberach' :22,
'LK Birkenfeld' :307,
'LK Bitburg-Pruem' :335,
'LK Bodenseekreis' :23,
'LK Boeblingen' :57,
'LK Boerde' :360,
'LK Borken' :296,
'LK Breisgau-Hochschwarzwald' :35,
'LK Burgenlandkreis' :352,
'LK Calw' :46,
'LK Celle' :240,
'LK Cham' :123,
'LK Cloppenburg' :253,
'LK Coburg' :112,
'LK Cochem-Zell' :308,
'LK Coesfeld' :298,
'LK Cuxhaven' :241,
'LK Dachau' :88,
'LK Dahme-Spreewald' :161,
'LK Darmstadt-Dieburg' :179,
'LK Deggendorf' :73,
'LK Diepholz' :233,
'LK Dillingen a.d.Donau' :134,
'LK Dingolfing-Landau' :74,
'LK Dithmarschen' :397,
'LK Donau-Ries' :135,
'LK Donnersbergkreis' :317,
'LK Dueren' :288,
'LK Ebersberg' :89,
'LK Eichsfeld' :413,
'LK Eichstaett' :90,
'LK Elbe-Elster' :162,
'LK Emmendingen' :36,
'LK Emsland' :255,
'LK Ennepe-Ruhr-Kreis' :9,
'LK Enzkreis' :47,
'LK Erding' :91,
'LK Erlangen-Hoechstadt' :32,
'LK Erzgebirgskreis' :374,
'LK Esslingen' :58,
'LK Euskirchen' :289,
'LK Forchheim' :114,
'LK Freising' :92,
'LK Freudenstadt' :48,
'LK Freyung-Grafenau' :75,
'LK Friesland' :256,
'LK Fuerstenfeldbruck' :93,
'LK Fuerth' :34,
'LK Fulda' :197,
'LK Garmisch-Partenkirchen' :94,
'LK Germersheim' :319,
'LK Giessen' :192,
'LK Gifhorn' :223,
'LK Goeppingen' :59,
'LK Goerlitz' :383,
'LK Goettingen' :225,
'LK Goslar' :224,
'LK Gotha' :416,
'LK Grafschaft Bentheim' :257,
'LK Greiz' :417,
'LK Gross-Gerau' :182,
'LK Guenzburg' :136,
'LK Guetersloh' :263,
'LK Hameln-Pyrmont' :234,
'LK Harburg' :242,
'LK Hassberge' :148,
'LK Havelland' :164,
'LK Heidenheim' :60,
'LK Heilbronn' :62,
'LK Heinsberg' :290,
'LK Helmstedt' :226,
'LK Herford' :264,
'LK Hersfeld-Rotenburg' :198,
'LK Herzogtum Lauenburg' :400,
'LK Hildburghausen' :418,
'LK Hildesheim' :236,
'LK Hochsauerlandkreis' :13,
'LK Hochtaunuskreis' :183,
'LK Hoexter' :265,
'LK Hof' :115,
'LK Hohenlohekreis' :63,
'LK Holzminden' :237,
'LK Ilm-Kreis' :419,
'LK Jerichower Land' :362,
'LK Kaiserslautern' :320,
'LK Karlsruhe' :50,
'LK Kassel' :199,
'LK Kelheim' :76,
'LK Kitzingen' :149,
'LK Kleve' :269,
'LK Konstanz' :38,
'LK Kronach' :117,
'LK Kulmbach' :118,
'LK Kusel' :322,
'LK Kyffhaeuserkreis' :421,
'LK Lahn-Dill-Kreis' :193,
'LK Landsberg a.Lech' :96,
'LK Landshut' :77,
'LK Leer' :258,
'LK Leipzig' :393,
'LK Lichtenfels' :119,
'LK Limburg-Weilburg' :194,
'LK Lindau' :139,
'LK Lippe' :266,
'LK Loerrach' :39,
'LK Ludwigsburg' :64,
'LK Ludwigslust-Parchim' :215,
'LK Luechow-Dannenberg' :243,
'LK Lueneburg' :244,
'LK Maerkisch-Oderland' :165,
'LK Maerkischer Kreis' :14,
'LK Main-Kinzig-Kreis' :184,
'LK Main-Spessart' :150,
'LK Main-Tauber-Kreis' :65,
'LK Main-Taunus-Kreis' :185,
'LK Mainz-Bingen' :325,
'LK Mansfeld-Suedharz' :354,
'LK Marburg-Biedenkopf' :195,
'LK Mayen-Koblenz' :310,
'LK Meissen' :386,
'LK Merzig-Wadern' :339,
'LK Mettmann' :274,
'LK Miesbach' :97,
'LK Miltenberg' :151,
'LK Minden-Luebbecke' :267,
'LK Muehldorf a.Inn' :98,
'LK Muenchen' :99,
'LK Neckar-Odenwald-Kreis' :53,
'LK Neu-Ulm' :141,
'LK Neuburg-Schrobenhausen' :101,
'LK Neumarkt i.d.OPf.' :124,
'LK Neunkirchen' :340,
'LK Neustadt a.d.Aisch-Bad Windsheim' :67,
'LK Neuwied' :311,
'LK Nienburg (Weser)' :238,
'LK Nordfriesland' :403,
'LK Nordhausen' :422,
'LK Nordwestmecklenburg' :213,
'LK Northeim' :227,
'LK Nuernberger Land' :69,
'LK Oberallgaeu' :142,
'LK Oberbergischer Kreis' :292,
'LK Oberhavel' :166,
'LK Oberspreewald-Lausitz' :167,
'LK Odenwaldkreis' :186,
'LK Oder-Spree' :168,
'LK Offenbach' :188,
'LK Oldenburg' :0,
'LK Olpe' :15,
'LK Ortenaukreis' :40,
'LK Osnabrueck' :2,
'LK Ostalbkreis' :17,
'LK Ostallgaeu' :143,
'LK Osterholz' :245,
'LK Ostholstein' :404,
'LK Ostprignitz-Ruppin' :169,
'LK Paderborn' :268,
'LK Passau' :79,
'LK Peine' :229,
'LK Pfaffenhofen a.d.Ilm' :102,
'LK Pinneberg' :405,
'LK Ploen' :406,
'LK Potsdam-Mittelmark' :170,
'LK Prignitz' :172,
'LK Rastatt' :55,
'LK Ravensburg' :24,
'LK Recklinghausen' :301,
'LK Regen' :81,
'LK Regensburg' :126,
'LK Rems-Murr-Kreis' :18,
'LK Rendsburg-Eckernfoerde' :407,
'LK Reutlingen' :25,
'LK Rhein-Erft-Kreis' :293,
'LK Rhein-Hunsrueck-Kreis' :312,
'LK Rhein-Kreis Neuss' :279,
'LK Rhein-Lahn-Kreis' :313,
'LK Rhein-Neckar-Kreis' :56,
'LK Rhein-Pfalz-Kreis' :328,
'LK Rhein-Sieg-Kreis' :294,
'LK Rheingau-Taunus-Kreis' :189,
'LK Rheinisch-Bergischer Kreis' :295,
'LK Rhoen-Grabfeld' :152,
'LK Rosenheim' :103,
'LK Rostock' :216,
'LK Rotenburg (Wuemme)' :246,
'LK Roth' :70,
'LK Rottal-Inn' :82,
'LK Rottweil' :41,
'LK Saale-Holzland-Kreis' :423,
'LK Saale-Orla-Kreis' :424,
'LK Saalekreis' :356,
'LK Saalfeld-Rudolstadt' :425,
'LK Saar-Pfalz-Kreis' :343,
'LK Saarlouis' :342,
'LK Saechsische Schweiz-Osterzgebirge' :389,
'LK Sankt Wendel' :344,
'LK Schaumburg' :239,
'LK Schleswig-Flensburg' :408,
'LK Schmalkalden-Meiningen' :426,
'LK Schwaebisch Hall' :19,
'LK Schwalm-Eder-Kreis' :201,
'LK Schwandorf' :128,
'LK Schwarzwald-Baar-Kreis' :42,
'LK Schweinfurt' :153,
'LK Segeberg' :409,
'LK Siegen-Wittgenstein' :16,
'LK Sigmaringen' :26,
'LK Soemmerda' :427,
'LK Soest' :260,
'LK Sonneberg' :428,
'LK Spree-Neisse' :173,
'LK Stade' :248,
'LK Stadtverband Saarbruecken' :341,
'LK Starnberg' :105,
'LK Steinburg' :410,
'LK Steinfurt' :302,
'LK Stendal' :367,
'LK Stormarn' :411,
'LK Straubing-Bogen' :83,
'LK Suedliche Weinstrasse' :330,
'LK Suedwestpfalz' :331,
'LK Teltow-Flaeming' :174,
'LK Tirschenreuth' :129,
'LK Traunstein' :106,
'LK Trier-Saarburg' :336,
'LK Tuebingen' :27,
'LK Tuttlingen' :43,
'LK Uckermark' :175,
'LK Uelzen' :249,
'LK Unna' :261,
'LK Unstrut-Hainich-Kreis' :430,
'LK Unterallgaeu' :144,
'LK Vechta' :3,
'LK Verden' :250,
'LK Viersen' :281,
'LK Vogelsbergkreis' :196,
'LK Vogtlandkreis' :378,
'LK Vorpommern-Greifswald' :214,
'LK Vorpommern-Ruegen' :212,
'LK Vulkaneifel' :338,
'LK Waldeck-Frankenberg' :202,
'LK Waldshut' :44,
'LK Warendorf' :303,
'LK Wartburgkreis' :431,
'LK Weilheim-Schongau' :107,
'LK Weimarer Land' :433,
'LK Weissenburg-Gunzenhausen' :72,
'LK Werra-Meissner-Kreis' :203,
'LK Wesel' :282,
'LK Wesermarsch' :4,
'LK Westerwaldkreis' :314,
'LK Wetteraukreis' :190,
'LK Wittenberg' :350,
'LK Wittmund' :6,
'LK Wolfenbuettel' :231,
'LK Wuerzburg' :155,
'LK Wunsiedel i.Fichtelgebirge' :120,
'LK Zollernalbkreis' :29,
'LK Zwickau' :379,
'Region Hannover' :235,
'SK Amberg' :122,
'SK Ansbach' :30,
'SK Aschaffenburg' :145,
'SK Augsburg' :132,
'SK Baden-Baden' :45,
'SK Bamberg' :108,
'SK Bayreuth' :110,
'SK Berlin Charlottenburg-Wilmersdorf' :157,
'SK Berlin Friedrichshain-Kreuzberg' :157,
'SK Berlin Lichtenberg' :157,
'SK Berlin Marzahn-Hellersdorf' :157,
'SK Berlin Mitte' :157,
'SK Berlin Neukoelln' :157,
'SK Berlin Pankow' :157,
'SK Berlin Reinickendorf' :157,
'SK Berlin Spandau' :157,
'SK Berlin Steglitz-Zehlendorf' :157,
'SK Berlin Tempelhof-Schoeneberg' :157,
'SK Berlin Treptow-Koepenick' :157,
'SK Bielefeld' :262,
'SK Bochum' :7,
'SK Bonn' :286,
'SK Bottrop' :297,
'SK Brandenburg a.d.Havel' :159,
'SK Braunschweig' :222,
'SK Bremen' :176,
'SK Chemnitz' :371,
'SK Coburg' :112,
'SK Cottbus' :160,
'SK Darmstadt' :180,
'SK Dessau-Rosslau' :348,
'SK Dortmund' :8,
'SK Dresden' :382,
'SK Duesseldorf' :271,
'SK Duisburg' :270,
'SK Emden' :254,
'SK Erfurt' :414,
'SK Erlangen' :33,
'SK Essen' :272,
'SK Flensburg' :398,
'SK Frankenthal' :318,
'SK Frankfurt (Oder)' :163,
'SK Frankfurt am Main' :181,
'SK Freiburg i.Breisgau' :37,
'SK Fuerth' :34,
'SK Gelsenkirchen' :299,
'SK Gera' :415,
'SK Hagen' :10,
'SK Halle' :353,
'SK Hamburg' :177,
'SK Hamm' :11,
'SK Heidelberg' :49,
'SK Heilbronn' :62,
'SK Herne' :12,
'SK Hof' :115,
'SK Ingolstadt' :95,
'SK Jena' :420,
'SK Kaiserslautern' :320,
'SK Karlsruhe' :50,
'SK Kassel' :199,
'SK Kaufbeuren' :137,
'SK Kempten' :138,
'SK Kiel' :399,
'SK Koblenz' :309,
'SK Koeln' :287,
'SK Krefeld' :273,
'SK Landau i.d.Pfalz' :323,
'SK Landshut' :77,
'SK Leipzig' :393,
'SK Leverkusen' :291,
'SK Ludwigshafen' :324,
'SK Luebeck' :401,
'SK Magdeburg' :363,
'SK Mainz' :326,
'SK Mannheim' :52,
'SK Memmingen' :140,
'SK Moenchengladbach' :275,
'SK Muelheim a.d.Ruhr' :276,
'SK Muenchen' :99,
'SK Muenster' :300,
'SK Neumuenster' :402,
'SK Neustadt a.d.Weinstrasse' :125,
'SK Nuernberg' :68,
'SK Oberhausen' :277,
'SK Offenbach' :188,
'SK Oldenburg' :0,
'SK Osnabrueck' :1,
'SK Passau' :79,
'SK Pforzheim' :54,
'SK Pirmasens' :330,
'SK Potsdam' :171,
'SK Regensburg' :126,
'SK Remscheid' :278,
'SK Rosenheim' :103,
'SK Rostock' :216,
'SK Salzgitter' :230,
'SK Schwabach' :71,
'SK Schweinfurt' :153,
'SK Schwerin' :218,
'SK Solingen' :280,
'SK Speyer' :329,
'SK Straubing' :84,
'SK Stuttgart' :20,
'SK Suhl' :429,
'SK Trier' :337,
'SK Ulm' :28,
'SK Weiden i.d.OPf.' :130,
'SK Weimar' :432,
'SK Wiesbaden' :191,
'SK Wilhelmshaven' :5,
'SK Wolfsburg' :232,
'SK Worms' :332,
'SK Wuerzburg' :155,
'SK Wuppertal' :283,
'SK Zweibruecken' :333,
'StadtRegion Aachen' :284,

}

In [ ]:
# Todo which Data is not used from Covid data 
# Todo which Landkreise can be combined from Geojson?
#which Landkreis is not used?

In [ ]:
# Get the summed up cases for provided date for every state and return it as a dataframe
def getCovidSumLaandkreis(date,input_covid_de_sorted):
    
    cases_of_state = []
    state_ids = []
    dates = []
    dates_str = []
    CasesPer100k = []
    LandkreiseShow = []
    
    #Check which Landkreis ID is used to combine Multible Landkreise e.g. Berlin
    #is in multible entries:
    all_ids = []
    double_ids = []
    for landkreis in LandkreiseDf:
        try:
            all_ids.append(GeoDictLandkreise[landkreis])
        except:
            continue

    double_ids = [item for item, count in collections.Counter(all_ids).items() if count > 1]
    #print(double_ids)

    
    
    for landkreis in LandkreiseDf:
        current_id = -1
        #Check if data is available
        try:
            current_id = GeoDictLandkreise[landkreis]
        except:
            #print('No Data For:', landkreis)
            continue
            
            
        # check if Landkreis id is already used in geo - if not 
        if GeoDictLandkreise[landkreis] in state_ids:
            continue 
            
        
        if current_id in double_ids: # combine Berlin
#             #only once for multible ids
#             double_ids.remove(current_id)
            
#             LandkreiseShow.append(landkreis)
#             state_ids.append(current_id)
#             dates.append(date)
#             dates_str.append(str(date.strftime("%Y-%m-%d")))
            
#             cases_of_state_multi = []
#             all_InhabitansLandkreis = []
            
            
#             for landkreis_multi in LandkreiseDf:
                
#                 try:
#                     current_id_multi = GeoDictLandkreise[landkreis]
#                 except:
#                     continue
                
#                 if current_id_multi == current_id:
#                     print('double: ',landkreis_multi)
#                     cases_of_state_multi.append(np.sum(input_covid_de_sorted.loc['{}'.format(date)].query("county == '{}'".format(landkreis))['cases']))
#                     all_InhabitansLandkreis.append(InhabitansLandkreiseDict[landkreis])
                
#             cases_of_state.append(sum(cases_of_state_multi))
#             CasesPer100k.append(sum(cases_of_state_multi)/sum(all_InhabitansLandkreis)*100000)
            
            #print('multi_id= ', current_id)
        #ToDo Put multible ids together
                
            continue
            
            
            
        
        else:
            #print('single_id= ',  current_id)
            LandkreiseShow.append(landkreis)
            state_ids.append(current_id)
            dates.append(date)
            dates_str.append(str(date.strftime("%Y-%m-%d")))
            
            cases_of_state.append(np.sum(input_covid_de_sorted.loc['{}'.format(date)].query("county == '{}'".format(landkreis))['cases']))
            CasesPer100k.append(np.sum(input_covid_de_sorted.loc['{}'.format(date)].query("county == '{}'".format(landkreis))['cases'])/InhabitansLandkreiseDict[landkreis]*100000)

    
    d = {'date': dates,'date_str': dates_str,'country': LandkreiseShow,'state_id' : state_ids, 'cases': cases_of_state,'NewCasesPer100k':CasesPer100k}
    df_germany = pd.DataFrame(data=d)
    return df_germany

In [ ]:
# firstmap = getCovidSumLaandkreis(datetime.date(2020, 3, 26),covid_de)

In [ ]:
# sum(covid_de.query("date == '2020-03-26'").query("state == 'Berlin'")['cases'])

In [ ]:
# firstmap

In [ ]:
# fig = px.choropleth(firstmap, geojson=geojson_Landkreise, color="NewCasesPer100k",
#                     locations="state_id",color_continuous_scale="Plasma",
#                     animation_frame ="date_str",
#                     range_color=[0,100],
#                     hover_data=['country','country']
                    
#                    )
# fig.update_geos(fitbounds="locations")
# fig.update_layout(title='Prototype',height=800)
# fig.show()


In [ ]:
# not used geo 

# notgeo = [x for x in range(434)]
# used = [
# 304,
# 273,
# 323,
# 77,
# 332,
# 155,
# 283,
# 333,
# 284,
# 1111111111111111111111111111111111111111111,# more from excel

# ]




In [ ]:
# notused_geopmap_id = []
# for ngeo in notgeo:
#     if ngeo not in used:
#         #print(ngeo)
#         notused_geopmap_id.append(ngeo)
        
# notused_geopmap_color = []
# for x in notused_geopmap:
#     notused_geopmap_color.append(1)

# LandkreiseGeoJson = [] 
# for id_notuesd in notused_geopmap_id:
#     for i, obj in enumerate(geojson_Landkreise['features']):
#         if i == id_notuesd:
#             print(i, obj['properties']['NAME_3'])
        
        

In [ ]:
# d_ = {'notused_geopmap': notused_geopmap,'notused_geopmap_color':notused_geopmap_color}# ,'NewCasesPer100k':CasesPer100k}
# not_geopmap_color = pd.DataFrame(data=d_)

# Get All the Data

In [ ]:
# create combined dataframe of all days
current_Landkreis_data=  getCovidSumLaandkreis(datetime.date(2020, 3, 26),covid_de)

start_date = datetime.date(2020, 3, 27)
end_date = datetime.date(2021, 1, 1)
delta = datetime.timedelta(days=1)

delta5 = datetime.timedelta(days=5)




while start_date <= end_date:
    start_date += delta

    # Data of the day
    time_data = getCovidSumLaandkreis(start_date,covid_de)
    time_data["All_Cases"] = 0
    time_data["7TageInzidenz"] = 0
    
    
    
    #Get The Amount of NewCases Per 7 Days
    for index, row in time_data.iterrows():
        # get all the previous cases
        time_data["7TageInzidenz"].iloc[index] = np.sum(current_Landkreis_data[current_Landkreis_data.date > row['date'] - pd.to_timedelta("7day")].query("country == '{}'".format(row['country'])) ['NewCasesPer100k'])

    
    current_Landkreis_data = current_Landkreis_data.append(time_data).reset_index(drop=True)

    print(start_date , end=' ')
    
    

In [ ]:
# use current_State_data as new output 
with open('./preprocessing/current_Landkreis_data.pickle', 'wb') as f:
    pickle.dump(current_Landkreis_data, f)

#os.mkdir('./preprocessing')

In [ ]:
# fig1 = px.choropleth(current_Landkreis_data[:50000], geojson=geojson_Landkreise, color="7TageInzidenz",
#                     locations="state_id",color_continuous_scale="Plasma",
#                     animation_frame ="date_str",
#                     range_color=[0,400],
#                     hover_data=['country','7TageInzidenz'],
                    
#                    )
# fig1.update_geos(fitbounds="locations")
# fig1.update_layout(title='Germany 7-day incidence map',height=800)
# fig1.show()

In [ ]:
# fig2 = px.choropleth(current_Landkreis_data[:50000], geojson=geojson_Landkreise, color="7TageInzidenz",
#                     locations="state_id",color_continuous_scale="Plasma",
#                     animation_frame ="date_str",
#                     range_color=[0,400],
#                     hover_data=['country','7TageInzidenz'],
                    
#                    )
# fig2.update_geos(fitbounds="locations")
# fig2.update_layout(title='Germany 7-day incidence map',height=800)
# fig2.show()

In [ ]:
len(current_Landkreis_data)
# len(np.union1d(np.array(covid_de_sorted['county']),[]))

In [ ]:
# np.sort(np.union1d(np.array(covid_de_sorted['county']),[]))

In [ ]:
# LandkreiseGeoJson = [] 
# for obj in geojson['features']:
#     LandkreiseGeoJson.append(obj['properties']['NAME_3'])
# len(LandkreiseGeoJson)

In [ ]:
#np.sort(LandkreiseGeoJson)

In [ ]:
 
# list(set(np.array(covid_de_sorted['county'])) & set(LandkreiseGeoJson))